# E7 Project

## 1) 데이터 준비와 전처리

In [126]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
using_cols = ['user_id', 'movie_id', 'rating']
ratings = ratings[using_cols]
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [127]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [128]:
ratings.rename(columns={'rating':'counts'}, inplace=True)

# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [129]:
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [130]:
def title_to_id(name):
    t = movies['title'] == name
    tt = movies[t]
    r = tt['movie_id'].values[0].item()
    return r

def id_to_title(_id):
    t = movies['movie_id'] == _id
    tt = movies[t]
    r = tt['title'].item()
    return r    

## 2) 데이터 분석

In [131]:
ratings['user_id'].nunique()

6039

In [132]:
ratings['movie_id'].nunique()

3628

In [133]:

movie_count = ratings.groupby('movie_id')['movie_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: movie_id, dtype: int64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [134]:
my_favorite = ['avengers, the (1998)' , 'lord of the rings, the (1978)' ,'saving private ryan (1998)' ,'shawshank redemption, the (1994)' ,'jurassic park (1993)']

my_favorite_id = []

for i in my_favorite:
    my_favorite_id.append(title_to_id(i))
    
print(my_favorite_id)

[2153, 2116, 2028, 318, 480]


In [135]:

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['junmo']*5, 'movie_id': my_favorite_id, 'counts':[5]*5})

if not ratings.isin({'user_id':['junmo']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,junmo,2153,5
1,junmo,2116,5
2,junmo,2028,5
3,junmo,318,5
4,junmo,480,5


In [136]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [137]:
print(user_to_idx['junmo'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(movie_to_idx[title_to_id('avengers, the (1998)')])

6039
948


In [138]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
print(len(temp_user_data), len(ratings))
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

836483 836483
user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,948,5
1,6039,926,5
2,6039,48,5
3,6039,157,5


## 4) CSR matrix를 직접 만들기

In [139]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

# print(len(df.counts), len(df.user_id), len(df.movie_id), num_user, num_movie)
csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시키기


In [140]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [141]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=1000, regularization=0.01, use_gpu=False, iterations=100, dtype=np.float32)

In [142]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose
als_model.fit(csr_data_transpose)

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [143]:
ans = []
for v in my_favorite_id:
    junmo, m = user_to_idx['junmo'], movie_to_idx[v]
    jm_vector, m_vector = als_model.user_factors[junmo], als_model.item_factors[m]
    ans.append(np.dot(jm_vector, m_vector))
    
print(ans)

[0.5717106, 0.8411533, 0.99169254, 0.9827534, 0.98937595]


In [144]:
junmo, m = user_to_idx['junmo'], movie_to_idx[title_to_id('toy story (1995)')]
jm_vector, m_vector = als_model.user_factors[junmo], als_model.item_factors[m]
np.dot(jm_vector, m_vector)

-0.011037377

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [145]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[2153,
 1383,
 706,
 3458,
 1102,
 814,
 2486,
 1553,
 2226,
 1514,
 1622,
 2242,
 3890,
 1107,
 1062]

In [146]:
def get_similar_movie(movie_name: str):
    m_id = title_to_id(movie_name)
    movie_id = movie_to_idx[m_id]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [147]:
a = get_similar_movie('avengers, the (1998)')
for i in a:
    print(id_to_title(i))

avengers, the (1998)
sunset park (1996)
blood and sand (sangre y arena) (1989)
american strays (1996)
boy called hate, a (1995)
adrenalin: fear the rush (1996)
24-hour woman (1998)
late bloomers (1996)
ring, the (1927)
kicked in the head (1997)


## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [148]:
user = user_to_idx['junmo']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(705, 0.11507117),
 (1549, 0.1139628),
 (1032, 0.10502137),
 (339, 0.10072114),
 (140, 0.09298943),
 (559, 0.088469855),
 (1943, 0.08844617),
 (49, 0.08401048),
 (1338, 0.08317249),
 (1478, 0.081492476),
 (920, 0.08016197),
 (515, 0.078675814),
 (1219, 0.07582837),
 (1815, 0.07512081),
 (710, 0.07452475),
 (891, 0.07419686),
 (2007, 0.07363501),
 (1779, 0.07296177),
 (526, 0.0728302),
 (1431, 0.07076108)]

In [149]:
a = [idx_to_movie[i[0]] for i in movie_recommended]
for i in a:
    print(id_to_title(i))

black cauldron, the (1985)
black hole, the (1979)
apostle, the (1997)
arthur (1981)
clear and present danger (1994)
robin hood (1973)
batman & robin (1997)
secret garden, the (1993)
harvey (1950)
johnny mnemonic (1995)
night of the comet (1984)
batman forever (1995)
first blood (1982)
pink floyd - the wall (1982)
mummy, the (1999)
adventures of buckaroo bonzai across the 8th dimension, the (1984)
inspector gadget (1999)
american werewolf in paris, an (1997)
superman (1978)
beyond rangoon (1995)


In [150]:
a = movie_to_idx[title_to_id('black hole, the (1979)')]
explain = als_model.explain(user, csr_data, itemid=a)

a = [(idx_to_movie[i[0]], i[1]) for i in explain[1]]
a = [idx_to_movie[i[0]] for i in a]
for i in a:
    print(id_to_title(i))

if.... (1968)
raise the titanic (1980)
rosencrantz and guildenstern are dead (1990)
jean de florette (1986)
grand day out, a (1992)


## Discussion

이번 노드는 이것저것 이해 할 수 없는 에러들이 너무 많이 발생해서 힘들었다.
CSR 만들 때 ROW, COL 인덱스가 뭐 크다 어쩌구 저쩌구 하는데 검색을 해봐도 별 다른 정보가 안나와서 처음부터 놓친 부분이 있나 검토하였다.
중간에 인덱싱을 재조정?하는 부분이 있는데 그 부분을 따로 안하고 해서 그런가 싶은데 정확한 이유는 모르겠다.
암튼 다시 차근차근 하니 잘 된다. 학습을 좀 빡세게 시켰더니 살짝 오버피팅 난 것 같지만 그래도 학습 성공!